In [2]:
from mip import *

In [3]:
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            linhas = arquivo.readlines()
            tamanho_da_lista = len(linhas)
            lista_restricoes = []
            for i in range(tamanho_da_lista):
                linha = linhas[i].split()
                match i:
                    case 0:
                        quantidade_variaveis = int(linha[0])
                        quantidade_restricoes = int(linha[1])
                    case 1:
                        coeficientes_objetivo = []
                        for variaveis in range(quantidade_variaveis):
                            coeficientes_objetivo.append(int(linha[variaveis]))    
                    case _:
                        tamanho_da_linha = len(linha)
                        lista = []
                        for restricao in range(tamanho_da_linha):
                            lista.append(int(linha[restricao]))
                        lista_restricoes.append(lista)

        # Retorna os dados obtidos do arquivo
        return (quantidade_variaveis, quantidade_restricoes, coeficientes_objetivo, lista_restricoes)
    
    except FileNotFoundError:
        print(f'O arquivo "{nome_arquivo}" não foi encontrado.')

In [4]:
def Criar_modelo(dados):
    quant_variaveis = dados[0]
    quant_restricoes = dados[1]
    coef_objetivo = dados[2]
    restricoes = dados[3]

    # Criação de um modelo de Maximização
    modelo = Model(sense=MAXIMIZE, solver_name=CBC)
    # Atribuição de variáveis contínuas no intervalo [0,1]
    x = {i: modelo.add_var(var_type=CONTINUOUS, name=f'x_{i}', lb=0.0, ub=1.0) for i in  range(1,quant_variaveis+1)}
    
    modelo.objective = xsum(coef_objetivo[i-1]*x[i] for i in range(1,quant_variaveis+1))

    for i in range(quant_restricoes):
        tamanho_lista = len(restricoes[i])
        soma =  xsum(restricoes[i][j]*x[j+1] for j in range (tamanho_lista-1))
        modelo += soma <= restricoes[i][tamanho_lista-1]

    return modelo

In [5]:
# Busca a variável mais próxima de 0.5
def who_is_closer(modelo):
    var_prox_05 = None
    melhor_dist = 1000

    for v in modelo.vars:
        dist = abs(v.x - 0.5)

        if dist < melhor_dist:
            var_prox_05 = v
            melhor_dist = dist
 
    return var_prox_05

In [6]:
# Verifica por arredondamento se elas são inteiras, ignorando a margem 1e-6
def todas_variaveis_inteiras(modelo):
    for var in modelo.vars:
        if abs(var.x - round(var.x)) >= 1e-6:
            return False
    return True

In [7]:
# Cria submodelos, variando para cada tipo de inteiro (0,1)
def criar_submodelo(modelo_referencial, var_fracionaria, valor_inteiro):
    # Cria uma cópia do modelo referencial
    submodelo = modelo_referencial.copy()

    # Adiciona a restrição para forçar a variável selecionada a ser inteira
    if valor_inteiro == 0:
        submodelo += (var_fracionaria <= valor_inteiro)
    elif valor_inteiro == 1:
        submodelo += (var_fracionaria >= valor_inteiro)

    return submodelo

In [8]:
# Função para resolver o problema por Fila (Busca em largura)
def branch_and_bound(model):

    # Definimos um valor para a inicialização
    melhor_solucao = -float("inf")
    # Variável que guarda o melhor modelo encontrado
    melhor_modelo = None
    # Inicializando uma fila com os modelos abertos
    fila = [model]

    # Só para quando a fila estiver vazia
    while fila:
        # Modelo atual é sempre o primeiro elemento da fila
        modelo_atual = fila[0]

        status = modelo_atual.optimize()

        # Verifica se houve uma solução viável
        if status == OptimizationStatus.OPTIMAL:

            # Busca a variável mais próxima de 0.5
            var_prox_05 = who_is_closer(modelo_atual)

            # Verifica se todas as variáveis são inteiras
            inteiro = todas_variaveis_inteiras(modelo_atual)
    
            if inteiro:
                # Sendo todas inteiras, vemos se ela é a melhor solução atual
                if modelo_atual.objective_value > melhor_solucao:
                    melhor_solucao = modelo_atual.objective_value
                    melhor_modelo = modelo_atual
        
            else:
                # Não sendo inteira, criamos novas condições para solucionar o problema
                if modelo_atual.objective_value > melhor_solucao:
        
                    model1 = criar_submodelo(modelo_atual, var_prox_05, 1)
                    model2 = criar_submodelo(modelo_atual, var_prox_05, 0)

                    # Adicionamos os novos modelos a lista 
                    fila.append(model1)
                    fila.append(model2)

        # Removemos o modelo corrente da fila
        fila.pop(0)
    
    return melhor_modelo

In [13]:
# Interação com todos os casos de teste
for i in range(1, 5):

    dados = ler_arquivo(f"teste{i}.txt")
    
    model = Criar_modelo(dados)

    solucao = branch_and_bound(model)

    # solucao.write(f"modelo{i}.lp") # salva modelo em arquivo
    # with open(f"modelo{i}.lp") as f: # lê e exibe conteúdo do arquivo
    #     print(f.read())

    print(f"Solution Teste {i}  = {solucao.objective_value:.2f}")
    print("Solution:")
    for v in solucao.vars:
        print(f"{v.name} = {v.x:.2f}")

    print("--------------------------------------------------")

Solution Teste 1  = 20.00
Solution:
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 1.00
x_6 = 1.00
x_7 = 0.00
--------------------------------------------------
Solution Teste 2  = 24.00
Solution:
x_1 = 0.00
x_2 = 0.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 1.00
x_7 = 1.00
x_8 = 0.00
x_9 = 1.00
--------------------------------------------------
Solution Teste 3  = 19.00
Solution:
x_1 = 0.00
x_2 = 0.00
x_3 = 1.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 1.00
x_8 = 0.00
x_9 = 0.00
--------------------------------------------------
Solution Teste 4  = 10.00
Solution:
x_1 = 0.00
x_2 = 0.00
x_3 = 1.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.00
x_9 = 0.00
--------------------------------------------------
